In [1]:
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
import pathlib 
import json
import imageio 
import pandas as pd 
#import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tqdm import tqdm
from moviepy.editor import  *
from collections import deque
%matplotlib inline

from sklearn.model_selection import train_test_split
# Run imports
from official.vision.configs import video_classification
from official.projects.movinet.configs import movinet as movinet_configs
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_layers
from official.projects.movinet.modeling import movinet_model
from official.projects.movinet.tools import export_saved_model
from tfrecords import parse_example




from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
tf.test.gpu_device_name()
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
    

In [3]:
ss_files = pathlib.Path('.') # current path

datafile_path = pathlib.Path('./')/'records/tfrecords' #records folder
datafile_path.mkdir(parents=True, exist_ok=True)


datafile_prefix = pathlib.Path('dataset/data')  #path to videos 

# we will save the files in the same place as our datafiles ()
tfrecords_save_path = datafile_path
train_path = tfrecords_save_path/'train/' # path to training shards 
val_path = tfrecords_save_path/'val/' # path to val shards 
test_path = tfrecords_save_path/'test/' # path to test shards 


In [4]:
def get_shards(path):
    shards = [str(path) for path in path.iterdir() if '.tfrecord' == path.suffix]
    return shards

train_shards=get_shards(train_path)
val_shards=get_shards(val_path)
test_shards=get_shards(test_path)


In [5]:

# create dataset with train shards
# customize adding: .map(your_tf_function) after '.map(parse_example)\'
# Note: sharding allows us to shuffle the data efficiently
def get_data(shards):
  dataset = tf.data.TFRecordDataset(shards)
  

  dataset=dataset.map(parse_example)
  #dataset=dataset.repeat(1000)
  dataset=dataset.batch(1)
  data=iter(dataset)
  #data=dataset.as_numpy_iterator()
  return data
#.shuffle(100)

In [6]:
train_dataset=get_data(train_shards)
test_dataset=get_data(test_shards)
val_dataset=get_data(val_shards)


In [7]:
train_dataset

In [ ]:
tf.compat.v1.data.get_output_types(train_dataset)

(tf.float32, tf.int64)

In [ ]:
tf.compat.v1.data.get_output_shapes(train_dataset)


(TensorShape([None, None, None, None, None]), TensorShape([None]))

In [ ]:
tf.compat.v1.data.get_output_classes(dataset)


(tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor)

# OLD Model

In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "signlanguage.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)
def create_model():
    model=Sequential()
    model.add(Input((None,224, 224, 3)))
    #model.add(Flatten())

    model.add(Conv3D(filters = 64, kernel_size = (1,3, 3), activation = 'relu'))
    model.add(Conv3D(filters = 64, kernel_size = (1,3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size = (1,2, 2)))
    model.add(GlobalAveragePooling3D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(2, activation = 'softmax'))

    model.summary()

    return model


In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "signlanguage.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)
def get_model(width=224, height=224, depth=None):
    """Build a 3D convolutional neural network model."""

    inputs = Input((depth,height, width,  3))

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)
    #x=Flatten()(x)

    #x = GlobalAveragePooling3D()(x)
    #x = Dense(units=512, activation="relu")(x)
    #x = Dropout(0.3)(x)
    x=Bidirectional(ConvLSTM2D(512,kernel_size=3))(x)
    

    outputs = Dense(units=2, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs, outputs, name="3dcnn")
    model.summary()
    return model


# MoVi net


In [7]:
model_id = 'a0'

tf.keras.backend.clear_session()

# Create backbone and model.
use_positional_encoding = model_id in {'a3', 'a4', 'a5'}
backbone = movinet.Movinet(
	model_id=model_id,
	causal=True,
	conv_type='2plus1d',
	se_type='2plus3d',
	activation='hard_swish',
	gating_activation='hard_sigmoid',
	use_positional_encoding=use_positional_encoding,
	use_external_states=True,
)

# Create a movinet classifier using this backbone.
model = movinet_model.MovinetClassifier(
	backbone,
	num_classes=600,
	output_states=True)

In [8]:
movinet_hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/stream/kinetics-600/classification/3'

movinet_hub_model = hub.KerasLayer(movinet_hub_url, trainable=True)


In [9]:
"""pretrained_weights = {w.name: w for w in movinet_hub_model.weights}

model_weights = {w.name: w for w in model.weights}

for name in pretrained_weights:
	model_weights[name].assign(pretrained_weights[name])"""


pretrained_weights = {w.name: w for w in movinet_hub_model.weights}
model_weights = {w.name: w for w in model.weights}

In [10]:
model = movinet_model.MovinetClassifier(
	backbone=backbone,
	num_classes=50,
	output_states=True)

In [12]:
# Input layer for the frame sequence
image_input = tf.keras.layers.Input(
    shape=[None, None, None, 3],
    dtype=tf.float32,
    name='image')

# Input layers for the different model states.
init_states_fn = movinet_hub_model.resolved_object.signatures['init_states']


state_shapes = {
    name: ([s if s > 0 else None for s in state.shape], state.dtype)
    for name, state in init_states_fn(tf.constant([0, 0, 0, 0, 3])).items()
}

states_input = {
    name: tf.keras.Input(shape[1:], dtype=dtype, name=name)
    for name, (shape, dtype) in state_shapes.items()
}

# Wrap the Movinet model in a Keras model so that it can be finetuned.

inputs = {**states_input, 'image': image_input}

outputs = model(inputs)

In [16]:
init_states_fn

<ConcreteFunction signature_wrapper(*, input_shape) at 0x230D8050F40>

In [ ]:
# This custom training step ignores the updated states during training as they are only important during inference.
class CustomModel(tf.keras.Model):
	def train_step(self, data):
		# Unpack the data. Its structure depends on your model and
		# on what you pass to `fit()`.
		if len(data) == 3:
			x, y, sample_weight = data
		else:
			sample_weight = None
			x, y = data
		with tf.GradientTape() as tape:
			pred, states = self(x, training=True)  # Forward pass
			
			# Compute the loss value
			# (the loss function is configured in `compile()`)
			loss = self.compiled_loss(y, pred, regularization_losses=self.losses, sample_weight=sample_weight)

		# Compute gradients
		trainable_vars = self.trainable_variables
		gradients = tape.gradient(loss, trainable_vars)
		
		# Update weights
		self.optimizer.apply_gradients(zip(gradients, trainable_vars))
		
		# Update metrics (includes the metric that tracks the loss)
		self.compiled_metrics.update_state(y, pred, sample_weight=sample_weight)
		
		# Return a dict mapping metric names to current value
		return {m.name: m.result() for m in self.metrics}
      
model = CustomModel(inputs, outputs, name='movinet')

In [ ]:
for layer in model.layers[:-1]:
	layer.trainable = False
model.layers[-1].trainable = True

In [ ]:
init_states = init_states_fn(tf.shape(tf.ones([1, 50, 224, 224, 3])))

In [ ]:
for layer in model.layers[:-1]:
	layer.trainable = False
model.layers[-1].trainable = True

In [ ]:
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [ ]:
history=model.fit(train_dataset ,epochs=1 )

ValueError: in user code:

    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Dell\AppData\Local\Temp\ipykernel_16328\215338575.py", line 12, in train_step
        pred, states = self(x, training=True)  # Forward pass
    File "d:\final final code\env\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\final final code\env\lib\site-packages\keras\engine\input_spec.py", line 200, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_n

In [ ]:
tf.keras.utils.plot_model(model, show_dtype=True, show_shapes=True)

In [15]:
model.summary()


Model: "movinet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, None, None,  0           []                               
                                 None, 3)]                                                        
                                                                                                  
 state/b0/l0/pool_buffer (Input  [(None, 1, 1, 1, 24  0          []                               
 Layer)                         )]                                                                
                                                                                                  
 state/b0/l0/pool_frame_count (  [(None,)]           0           []                               
 InputLayer)                                                                                

In [ ]:
model_id = 'a0'
tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
model = movinet_model.MovinetClassifier(
backbone=backbone,
num_classes=600)
model.build([batch_size, num_frames, resolution, resolution, 3])

movinet_hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/base/kinetics-600/classification/3'

movinet_hub_model = hub.KerasLayer(movinet_hub_url, trainable=True)
pretrained_weights = {w.name: w for w in movinet_hub_model.weights}
model_weights = {w.name: w for w in model.weights}

for name in pretrained_weights:
    model_weights[name].assign(pretrained_weights[name])

model = movinet_model.MovinetClassifier(
backbone=backbone,
num_classes=num_classes)
model.build([batch_size, num_frames, resolution, resolution, 3])

for layer in model.layers[:-2]:
    layer.trainable = True

train_steps = num_examples['train'] // batch_size
total_train_steps = train_steps * num_epochs
test_steps = num_examples['test'] // batch_size

loss_obj = tf.keras.losses.CategoricalCrossentropy(
from_logits=True,
label_smoothing=0.1)

metrics = [
tf.keras.metrics.TopKCategoricalAccuracy(
k=1, name='top_1', dtype=tf.float32),
tf.keras.metrics.TopKCategoricalAccuracy(
k=2, name='top_2', dtype=tf.float32),
]

initial_learning_rate = lr
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
initial_learning_rate, decay_steps=total_train_steps,
)
optimizer = tf.keras.optimizers.RMSprop(
learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

checkpoint_filepath = os.path.join('chkpnts', 'weights.{epoch:02d}-{val_loss:.2f}.hdf5')
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_filepath,
save_weights_only=False,
monitor='val_accuracy',
mode='max',
save_best_only=False,
verbose=1)

callbacks = [
tf.keras.callbacks.TensorBoard(log_dir='./logs'),
model_checkpoint_callback,
]

model.summary()